In [15]:
import ast

#base example of the node visitor
class v(ast.NodeVisitor):
   def generic_visit(self, node):
    print (type(node).__name__) #prints the type of value that is seen in the node
    ast.NodeVisitor.generic_visit(self, node) #generic visit to the node(iterates) (recursive)

x = v() #x is the function call
t = ast.parse('d[x] += v[y, x]') #example of a bit of code we are testing
x.visit(t) #call to generic_visit function

Module
AugAssign
Subscript
Name
Load
Name
Load
Store
Add
Subscript
Name
Load
Tuple
Name
Load
Name
Load
Load
Load


In [2]:
#example of node visitor that doesn't generic visit
#This means that the Load node isn't printed 

class w(v):
    def visit_Load(self, node): pass

y = w()
y.visit(t)

Module
AugAssign
Subscript
Name
Name
Store
Add
Subscript
Name
Tuple
Name
Name


In [3]:
'''This example is the same as the above code, but we are finding the actual names of
the values that are stored within the nodes themselves
'''
class z(v):
    def visit_Name(self, node): print ('Name:', node.id)

z().visit(t)

Module
AugAssign
Subscript
Name: d
Name: x
Store
Add
Subscript
Name: v
Tuple
Name: y
Name: x
Load
Load


In [4]:
'''This is a combination, so instead of not using generic visit, we use it and 
store all values that it finds into a vector (Something we need for comparing)
'''
class allnames(ast.NodeVisitor):
    def visit_Module(self, node):
        self.names = set()
        self.generic_visit(node)
        print (sorted(self.names))
    def visit_Name(self, node):
        self.names.add(node.id)

allnames().visit(t)

['d', 'v', 'x', 'y']


In [9]:
'''if we want an output that more low level for the computer, we can opt for this 
route. This could also help with future comparing'''

def str_node(node):
    if isinstance(node, ast.AST):
        fields = [(name, str_node(val)) for name, val in ast.iter_fields(node) if name not in ('left', 'right')]
        rv = '%s(%s' % (node.__class__.__name__, ', '.join('%s=%s' % field for field in fields))
        return rv + ')'
    else:
        return repr(node)
def ast_visit(node, level=0):
    print('  ' * level + str_node(node))
    for field, value in ast.iter_fields(node):
        if isinstance(value, list):
            for item in value:
                if isinstance(item, ast.AST):
                    ast_visit(item, level=level+1)
        elif isinstance(value, ast.AST):
            ast_visit(value, level=level+1)


ast_visit(ast.parse('a * y'))

Module(body=[<ast.Expr object at 0x7f0c983870a0>], type_ignores=[])
  Expr(value=BinOp(op=Mult()))
    BinOp(op=Mult())
      Name(id='a', ctx=Load())
        Load()
      Mult()
      Name(id='y', ctx=Load())
        Load()


In [16]:
#attempting to parse a function in a different file
# Step 1: Read the source file
with open('test_code/battle_generator.py', 'r') as file:
    other_module_content = file.read()

# Step 2: Parse the content to get an AST
module_ast = ast.parse(other_module_content)

# Step 3: Navigate to the function
# for node in ast.walk(module_ast):
#     if isinstance(node, ast.FunctionDef) and node.name == 'format':
#         function_ast = node
#         break

# Now, function_ast is the AST of my_function
# print(ast.dump(function_ast, indent=4))


# class v(ast.NodeVisitor):
#    def generic_visit(self, node):
#     print (type(node).__name__) #prints the type of value that is seen in the node
#     ast.NodeVisitor.generic_visit(self, node) #generic visit to the node(iterates) (recursive)

# x = v() #x is the function call
# t = ast.parse('d[x] += v[y, x]') #example of a bit of code we are testing
# x.visit(t) #call to generic_visit function

def str_node(node):
    if isinstance(node, ast.AST):
        fields = [(name, str_node(val)) for name, val in ast.iter_fields(node) if name not in ('left', 'right')]
        rv = '%s(%s' % (node.__class__.__name__, ', '.join('%s=%s' % field for field in fields))
        return rv + ')'
    else:
        return repr(node)
def ast_visit(node, level=0):
    print('  ' * level + str_node(node))
    for field, value in ast.iter_fields(node):
        if isinstance(value, list):
            for item in value:
                if isinstance(item, ast.AST):
                    ast_visit(item, level=level+1)
        elif isinstance(value, ast.AST):
            ast_visit(value, level=level+1)


ast_visit(module_ast)

Module(body=[<ast.Import object at 0x7f0c88cdbfa0>, <ast.Import object at 0x7f0c88cdbf40>, <ast.Import object at 0x7f0c88cdbee0>, <ast.Assign object at 0x7f0c88cdbe80>, <ast.Assign object at 0x7f0c88cdbd30>, <ast.Assign object at 0x7f0c88cdbbb0>, <ast.FunctionDef object at 0x7f0c88cdba30>, <ast.Assign object at 0x7f0c88cdae00>, <ast.Assign object at 0x7f0c88cdad10>, <ast.Assign object at 0x7f0c88cdab90>, <ast.Assign object at 0x7f0c88cda800>, <ast.Assign object at 0x7f0c88cda470>, <ast.For object at 0x7f0c88cda260>, <ast.Assign object at 0x7f0c88db6aa0>, <ast.With object at 0x7f0c88db6920>, <ast.Expr object at 0x7f0c88db66b0>], type_ignores=[])
  Import(names=[<ast.alias object at 0x7f0c88cdbf70>])
    alias(name='json', asname=None)
  Import(names=[<ast.alias object at 0x7f0c88cdbf10>])
    alias(name='os', asname=None)
  Import(names=[<ast.alias object at 0x7f0c88cdbeb0>])
    alias(name='string', asname=None)
  Assign(targets=[<ast.Name object at 0x7f0c88cdbe50>], value=Call(func=At

In [2]:
import ast

#testing profiling for functions
#Function for finding if functions within a python file have a return statement
def extract_return_info(function_node):
    """Extract return information from a function node."""
    returns = []
    for node in ast.walk(function_node):
        if isinstance(node, ast.Return):
            if node.value is None:
                returns.append('None')
            else:
                returns.append(ast.dump(node.value))
    return returns


def finding_constants(function_node):
    constants = []
    for node in ast.walk(function_node):
        if isinstance(node, ast.Constant):
            if node.value is None:
                constants.append('None')
            else:
                constants.append(node.value)
    return constants

def list_extraction(function_node):
    lst = []
    for node in ast.walk(function_node):
        if isinstance(node, ast.List):
            if node is None:
                lst.append('None')
            else:
                lst.append(ast.dump(node.id))

    return lst


#function that extracts info found and stores it into a dictionary
def extract_function_info(function_node):
    """Extract information from a function node."""
    function_name = function_node.name
    num_args = len(function_node.args.args)
    return_info = extract_return_info(function_node)
    constants = finding_constants(function_node)
    lst = list_extraction(function_node)
    # Add more attributes if needed
    return {
        'name': function_name,
        'num_args': num_args,
        'returns': return_info,
        'constants': constants,
        'lists': lst
    }

#creatinga  profile and storing them in a list
def profile_functions_from_file(file_path):
    """Profile functions in a given Python script."""
    with open(file_path, 'r') as file:
        content = file.read()

    # Parse the file content into an AST
    parsed_ast = ast.parse(content)

    # Traverse the AST and find function definitions
    functions_profile = []
    for node in ast.walk(parsed_ast):
        if isinstance(node, ast.FunctionDef):
            func_info = extract_function_info(node)
            functions_profile.append(func_info)

    return functions_profile

# Example usage
file_path = 'test_code/normalization.py'  # Replace with your script's file path
profile = profile_functions_from_file(file_path)
print(profile)

[{'name': 'normalize', 'num_args': 3, 'returns': ["Call(func=Attribute(value=Name(id='xarray_tree', ctx=Load()), attr='map_structure', ctx=Load()), args=[Name(id='normalize_array', ctx=Load()), Name(id='values', ctx=Load())], keywords=[])", "Name(id='array', ctx=Load())"], 'constants': ['Normalize variables using the given scales and (optionally) locations.', 'None', 'None', "Can't look up normalization constants because array has no name.", 'No normalization scale found for %s', 'No normalization location found for %s'], 'lists': []}, {'name': 'unnormalize', 'num_args': 3, 'returns': ["Call(func=Attribute(value=Name(id='xarray_tree', ctx=Load()), attr='map_structure', ctx=Load()), args=[Name(id='unnormalize_array', ctx=Load()), Name(id='values', ctx=Load())], keywords=[])", "Name(id='array', ctx=Load())"], 'constants': ['Unnormalize variables using the given scales and (optionally) locations.', 'None', 'None', "Can't look up normalization constants because array has no name.", 'No nor